In [83]:
import numpy as np
import pandas as pd
from spacy.lang.fr.stop_words import STOP_WORDS

In [84]:
df = pd.read_csv("data/Keyword_with_teeft_queneau_frprotocol_V2.csv")
df.head()

,Unnamed: 0,Unnamed: 0.1,title,texte,texte_keyword,keyword_without_ent,keyword_without_stop_word,keywords_lematisations
0,0,0,Notations.,"Dans l'S, à une heure d'affluence. Un type dan...",vingt-six ans chapeau mou$ruban cou$place libre,vingt-six$ans$chapeau$mou$ruban$cou$place$libre,vingt-six$ans$chapeau$mou$ruban$cou$place$libre,vingt-six$an$chapeau$mou$ruban$cou$place$libre
1,1,1,En partie double.,"Vers le milieu de la journée et à midi, je me ...",terrasse arrière$jeune homme$vieil adolescent$...,terrasse$arrière$jeune$homme$vieil$adolescent$...,terrasse$arrière$jeune$homme$vieil$adolescent$...,terrasse$arrière$jeune$homme$vieux$adolescent$...
2,2,2,Litotes.,Nous étions quelques-uns à nous déplacer de co...,jeune homme$deux heures,jeune$homme$deux$heures,jeune$homme$heures,jeune$homme$heure
3,3,3,Métaphoriquement.,"Au centre du jour, jeté dans le tas des sardin...",sardines voyageuses$grosse carapace blanche$gr...,sardines$voyageuses$grosse$carapace$blanche$gr...,sardines$voyageuses$grosse$carapace$blanche$gr...,sardine$voyageur$gros$carapace$blanc$grand$cou...
4,4,4,Rétrograde.,"Tu devrais ajouter un bouton à ton pardessus, ...",autre voyageur$jeune homme$chapeau ridicule,autre$voyageur$jeune$homme$chapeau$ridicule,voyageur$jeune$homme$chapeau$ridicule,autre$voyageur$jeune$homme$chapeau$ridicule


In [85]:
df=df.dropna()
df=df.reset_index(drop=True)

In [86]:
keys =[key for key in df.keys() if "keyword" in key]
dict_matrice ={}
for key in keys:
    keywords = "$".join(df[key].tolist())

    symbols = "!\"#%&()*+-./:;<=>?@[\]^_`{|}~\n,."
    for i in symbols:
        keywords.replace(i," ")

    transit_list_keywords=keywords.split('$')
    list_keywords=list()
    for keyword_list in transit_list_keywords:
        for word in keyword_list.split(" "):
            list_keywords.append(word)
    #list_keywords=keywords.split(' ')
    list_keywords=list(set(list_keywords))
    #list_keywords = [word for word in list_keywords if len(word)>1]
    list_keywords = list(set(list_keywords))
    dict_matrice[key]=list_keywords


for key in keys :
    print(len(dict_matrice[key]))

489
432
459
427


In [87]:
def mytokenizer(text):
    return text.split()
from sklearn.feature_extraction.text import CountVectorizer
corpus=df["texte"]
dic_vectors = {}

for key in keys:
    vocab = dict_matrice[key]
    vectorizer = CountVectorizer(vocabulary=vocab, tokenizer = mytokenizer)
    X = vectorizer.fit_transform(corpus)
    vectorizer.get_feature_names()
    dic_vectors[key] =  X.toarray()



c:\users\le muletier\pycharmprojects\test-de-quenaud\venv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\users\le muletier\pycharmprojects\test-de-quenaud\venv\lib\site-packages\sklearn\feature_extraction\text.py:1323: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  "Upper case characters found in"


In [88]:
from numpy.linalg import norm

In [89]:

df_corélations_cos= pd.DataFrame()


for key in keys:
    for index , vector in enumerate(dic_vectors[key]):
        df_corélations_cos.loc[index,"title"]=df.loc[index,"title"]
        df_corélations_cos.loc[index, "teeft_corrélation_"+str(key)] = np.dot(dic_vectors[key][0],vector)/(norm(dic_vectors[key][0])*norm(vector))



c:\users\le muletier\pycharmprojects\test-de-quenaud\venv\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys


In [90]:
df_corélations_cos.head()

,title,teeft_corrélation_texte_keyword,teeft_corrélation_keyword_without_ent,teeft_corrélation_keyword_without_stop_word,teeft_corrélation_keywords_lematisations
0,Notations.,1.000000,1.000000,1.000000,1.000000
1,En partie double.,0.198030,0.201008,0.207390,0.271052
2,Litotes.,0.000000,0.000000,0.000000,0.178174
3,Métaphoriquement.,0.083333,0.083333,0.091287,0.084515
4,Rétrograde.,0.273861,0.273861,0.288675,0.267261


In [91]:
df_corélations_cos.to_csv("results/result_teeft_correlation.csv")